In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pydataset import data

## 1. Attendance Data

Load the attendance.csv file and calculate an attendnace percentage for each student. 
One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [2]:
attendance = pd.read_csv('untidy-data/attendance.csv')
attendance

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
attendance = attendance.rename(columns={'Unnamed: 0': 'Name'})
attendance

,Name,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [4]:
attendance_melt = attendance.melt(id_vars='Name',
               var_name='day',
               value_name='status')
attendance

,Name,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [7]:
attendance = attendance.replace(['P','A','T','H'],[1, 0, 0.9, 0.5])
attendance.set_index(keys='Name', inplace=True)
attendance

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
Name,,,,,,,,
Sally,1,0.9,0.9,0.5,1.0,0.0,0.9,0.9
Jane,0,1.0,0.9,0.9,0.9,0.9,0.0,0.9
Billy,0,0.9,0.0,0.0,0.5,0.9,1.0,0.9
John,1,0.9,0.5,1.0,1.0,0.9,1.0,1.0


In [8]:
attendance.mean(axis=1)

Name
Sally    0.7625
Jane     0.6875
Billy    0.5250
John     0.9125
dtype: float64

## 2. Coffee Levels

- a. Read the coffee_levels.csv file.
- b. Transform the data so that each carafe is in it's own column.
- c. Is this the best shape for the data?

In [9]:
coffee = pd.read_csv('untidy-data/coffee_levels.csv')
coffee.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


In [10]:
coffee.shape

(30, 3)

In [22]:
coffee.pivot_table(index='hour', columns='coffee_carafe', values='coffee_amount')

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


I believe that this pivot table is is the most readable in this particular format if readability is the main goal.

## 3. Cake Recipes

- a. Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.
- b. Tidy the data as necessary.
- c. Which recipe, on average, is the best? recipe b
- d. Which oven temperature, on average, produces the best results? 275
- e. Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [ ]:
cakes = pd.read_csv('untidy-data/cake_recipes.csv')
cakes.head()

In [ ]:
cakes[['recipe','position']] = cakes['recipe:position'].str.split(':', expand=True)
cakes.drop(columns={'recipe:position'})

In [ ]:
cakes_tidy = cakes.pivot_table(index=['recipe','position']).reset_index()

In [ ]:
cakes_tidy['avg_score'] = (cakes_tidy['225'] + cakes_tidy['250'] + cakes_tidy['275'] + cakes_tidy['300']) / 4
cakes_tidy.info()

In [ ]:
#recipe with highest average score

cakes_tidy.recipe[cakes_tidy['avg_score'].idxmax()]

In [ ]:
# melt the tidy table so that the temperatures are in the same column
cakes_melt = cakes_tidy.melt(id_vars=['recipe', 'position'])
cakes_melt.rename(columns={'variable': 'oven_temp', 'value': 'score'}, inplace=True)

In [ ]:
# group by the oven temp variable so we can get the mean scores of each oven temp
# find the index value (oven temp) with the highest average score using idxmax
cakes_melt.groupby('oven_temp').mean().idxmax()

In [ ]:
# idxmax() returns the index that holds the max score
# loc() using the result of idxmax() and gives the observation with the highest score
    # recipe: b, position: bottom, oven: temp 300, score: 99.248541
cakes_melt.loc[cakes_melt.score.idxmax()]